In [ ]:
# Import the packages
import pandas as pd
# import bigquery
from google.cloud import bigquery
# from google.cloud import bigquery_storage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", 100)

In [ ]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [ ]:
# Declare some global inputs
entity_id = "PY_CL"
test_names = ["'CL_20221201_R_B0_O_ElasticityCLRestaurants'"]
ld_lb_tg = "Low basket, low distance" # Low distance low basket clsuter
hd_lb_tg = "Low basket, high distance" # High Distance low basket cluster
central_tg = "Central cluster" # Central cluster
ld_hb_tg = "High basket, low distance" # Low distance high basket cluster
hd_hb_tg = "High basket, high distance" # High distance high basket cluster
tier_1 = "1"
tier_2 = "2"
tier_3 = "3"
tier_4 = "4"
tier_5 = "5"
test_start_date = "2022-12-01"
test_end_date = "2023-01-12"
scheme_ids = ["'634,635'"]
num_bootstrap_samples = 1000 # Number of data points to have on the histogram
cl = 0.95
left_threshold = round((1 - cl) / 2, 4)
right_threshold = round((1 - (1 - cl) / 2), 4)

## QUERY

In [ ]:
test_query = """with  load_scheme_data as (
        SELECT 
            entity_id
            , scheme_id
            , scheme_active_from
            , IFNULL(scheme_active_to, CURRENT_TIMESTAMP()) scheme_active_to
            , scheme_component_configs.travel_time_config
        FROM `fulfillment-dwh-production.cl.pricing_configuration_versions`
)

 ##### LOAD ORDER DATA AND JOIN PM DATA 

, dps_order_raw as (
    SELECT
        -- Identifiers and supplementary fields     
        -- Date and time
        a.created_date AS created_date_utc,
        a.order_placed_at,
        -- Location of order
        -- a.region,
        a.entity_id,
        -- a.country_code,
        -- a.city_name,
        -- a.city_id,
        -- a.zone_name,
        -- a.zone_id,
        -- Order/customer identifiers and session data
        a.variant,
        -- a.experiment_id AS test_id,
        -- b.test_name,
        a.scheme_id,
        -- a.vendor_price_scheme_type,	-- The assignment type of the scheme to the vendor during the time of the order, such as "Automatic", "Manual", "Campaign", and "Country Fallback".
        -- Vendor data and information on the delivery
        a.vendor_id,
        a.dps_travel_time,
        e.cluster entity_cluster,
        v.cluster area_cluster,
        -- b.target_group AS target_group_bi,
        -- a.is_in_treatment,
        -- a.chain_id,
        -- a.chain_name,
        -- a.vertical_type,
        -- a.delivery_status,
        -- a.is_own_delivery,
        -- a.exchange_rate
        -- Business KPIs (These are the components of profit)
        a.platform_order_code platform_order_code,
        a.dps_travel_time_fee_local,
        a.dps_delivery_fee_local dps_delivery_fee_local,
        --avg(a.dps_travel_time_fee_local) dps_travel_time_fee_local,
        a.dps_surge_fee_local dps_surge_fee_local,
        a.delivery_fee_local delivery_fee_local,
        a.gfv_local gfv_local,
        a.gmv_local gmv_local
    from `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders_v2` a
    LEFT JOIN `fulfillment-dwh-production.cl.dps_ab_test_orders_v2` b ON a.entity_id = b.entity_id AND a.order_id = b.order_id
    LEFT JOIN `logistics-data-storage-staging.long_term_pricing.vendors_clustered_entity` e on a.entity_id = e.entity_id and a.vendor_id = e.vendor_id 
    LEFT JOIN `logistics-data-storage-staging.long_term_pricing.vendors_clustered` v on v.entity_id = a.entity_id and a.vendor_id = v.vendor_id
    where a.created_date between DATE("{test_start_date}") and DATE("{test_end_date}")
    AND b.test_name IN ({test_names})
    AND a.is_sent
    AND a.scheme_id IS NOT NULL
    AND a.entity_id = ("{entity_id}")
    AND a.scheme_id in (634,635)
    AND a.is_sent -- Successful orders
    AND a.is_in_treatment = true 
    AND a.is_own_delivery -- OD orders only
    and e.cluster is not null
    and v.cluster is not null
    AND a.variant != "Original" -- Exclude orders from ASAs
    AND a.is_match_experiment_vertical
    AND e.cluster not in ('Insufficient data')
    AND v.cluster not in ('Insufficient data')
)


, dps_order_with_pm as (
    SELECT 
    dps.*
    , sch.* EXCEPT(entity_id, scheme_id, scheme_active_from, scheme_active_to)
    , (SELECT MIN(tier) + 1 as tier
        FROM UNNEST(travel_time_config) tt
        WITH OFFSET as tier 
        WHERE dps_travel_time <= IFNULL(tt.travel_time_threshold,9999) 
    ) AS tt_tier
    from dps_order_raw dps
    LEFT JOIN load_scheme_data sch
      ON dps.entity_id = sch.entity_id
      AND dps.scheme_id = sch.scheme_id
      AND order_placed_at >= scheme_active_from 
      AND order_placed_at < scheme_active_to 
)
  SELECT
  a.created_date_utc,
  a.variant,
  -- Vendor data and information on the delivery
  a.vendor_id,
  a.entity_cluster,
  a.area_cluster,
  -- Business KPIs (These are the components of profit)
  a.platform_order_code platform_order_code,
  a.dps_travel_time_fee_local,
  a.dps_delivery_fee_local dps_delivery_fee_local,
  --avg(a.dps_travel_time_fee_local) dps_travel_time_fee_local,
  a.dps_surge_fee_local dps_surge_fee_local,
  a.delivery_fee_local delivery_fee_local,
  a.gfv_local gfv_local,
  a.gmv_local gmv_local,
  cast(a.tt_tier as string) tt_tier
from dps_order_with_pm as a
 """.format(test_names=', '.join(test_names), entity_id=entity_id, test_start_date=test_start_date, test_end_date=test_end_date)
# Execute the orders query
df_test_data = pd.read_gbq(test_query, project_id='logistics-customer-staging', dialect='standard')##, credentials=credentials)
df_test_data.head()

In [ ]:
df_test_data['delivery_fee_local'] = df_test_data['delivery_fee_local'].apply(lambda x: float(x))

In [ ]:
df_test_data['gfv_local'] = df_test_data['gfv_local'].apply(lambda x: float(x))

In [ ]:
df_test_data['df_afv'] = (df_test_data['delivery_fee_local'] / df_test_data['gfv_local'])*100

## TRY CLUSTERING BY ENTITY CLUSTER (OMAR CODE)

In [ ]:


# Run the bootstrapping function sequentially
vendor_labels = ["Low Basket, low Distance", "Low basket, high distance", "Central cluster", "High basket, low distance", "High basket, high distance"]
price_tiers = ["1","2","3","4", "5"]
sim_results = []
sim_counter = 1
for i in test_names:
    for j in vendor_labels:
        for l in price_tiers:
            df_sub_test = df_test_data[(df_test_data["area_cluster"] == j) & (df_test_data["tt_tier"] == l)]
            
            # Elasticity calculated by pct difference in no. of transactions / pct difference in average DF
            for k in range(1, num_bootstrap_samples + 1):
                print(f"Iteration {sim_counter}. Sampling with the following parameters --> vendor_group_label: {j}, price_tier:{l}, sample_num: {k}")
                df_ctl_sample = df_sub_test[df_sub_test["variant"] == "Control"].sample(frac=1, replace=True)
                df_var_sample = df_sub_test[df_sub_test["variant"] == "Variation1"].sample(frac=1, replace=True)
                # df_ctl_sample = df_ctl.sample(frac = 1, replace = True)            
                # df_var_sample = df_var.sample(frac=1, replace = True)
                num_orders_ctl = df_ctl_sample["platform_order_code"].nunique()
                num_orders_var = df_var_sample["platform_order_code"].nunique()

                    # avg_df_ctl = df_ctl_sample["delivery_fee_local"].avg()
                    # avg_df_var = df_var_sample["delivery_fee_local"].avg()
                avg_df_ctl = df_ctl_sample["delivery_fee_local"].sum() / num_orders_ctl
                avg_df_var = df_var_sample["delivery_fee_local"].sum() / num_orders_var

                
                

                pct_diff_orders = float((num_orders_var - num_orders_ctl) / num_orders_ctl)
                pct_diff_avg_df = float((avg_df_var - avg_df_ctl) / avg_df_ctl)

                elasticity = pct_diff_orders / pct_diff_avg_df
                avg_df_afv_var = float(df_var_sample["delivery_fee_local"].sum()/df_var_sample["gfv_local"].sum())

                output_dict = {
                    "test_name": 'CL_20221201_R_B0_O_ElasticityCLRestaurants',
                    "vendor_group_label": j,
                    "price_tier": l,
                    "sample_num": k,
                    "elasticity": elasticity,
                    "pct_diff_orders": pct_diff_orders,
                    "pct_diff_avg_df": pct_diff_avg_df,
                    "df_afv": avg_df_afv_var
                }

                sim_results.append(output_dict)
                    
                    # Increment the sim counter
                sim_counter += 1

df_sim_results = pd.DataFrame(sim_results)

## Try Area clustering modularized (Tincho proposal)

In [ ]:
df_control = df_test_data[df_test_data['variant']=="Control"]
df_variant_1 = df_test_data[df_test_data['variant']=='Variation1']

df_control


In [ ]:
# Run the bootstrapping function sequentially
vendor_labels = ["Low Basket, low Distance", "Low basket, high distance", "Central cluster", "High basket, low distance", "High basket, high distance"]
price_tiers = ["1","2","3","4", "5"]
sim_results = []
sim_counter = 1
for i in test_names:
    for j in vendor_labels:
        #     # Further filter the data based on the vendor labels
        print(j)
        if j == "Low Basket, low Distance":
            df_var = df_variant_1[df_variant_1["entity_cluster"] == ld_lb_tg]
            df_ctl = df_control[df_control["entity_cluster"] == ld_lb_tg]
           # print(df_ctl)
        elif j == "Low Basket, high Distance":
            df_var = df_variant_1[df_variant_1["entity_cluster"] == hd_lb_tg]
            df_ctl = df_control[df_control["entity_cluster"] == hd_lb_tg]
        elif j == "Central cluster":
            df_var = df_variant_1[df_variant_1["entity_cluster"] == central_tg]
            df_ctl = df_control[df_control["entity_cluster"] == central_tg]
        elif j == "Low distance, high basket":
            df_var = df_variant_1[df_variant_1["entity_cluster"] == ld_hb_tg]
            df_ctl = df_control[df_control["entity_cluster"] == ld_hb_tg]
        elif j == "High Basket, high distance":
            df_var = df_variant_1[df_variant_1["entity_cluster"]== hd_hb_tg]
            df_ctl = df_control[df_control["entity_cluster"] == hd_hb_tg]
        
        #  # Further filter the data based on price tiers
        for l in price_tiers:
            print(l)
            if l == "1st tier":
                df_var = df_variant_1[df_variant_1["tt_tier"] == tier_1]
                df_ctl = df_control[df_control["tt_tier"] == tier_1]
            # print(df_ctl)
            elif k == "2nd tier":
                df_var = df_variant_1[df_variant_1["tt_tier"] == tier_2]
                df_ctl = df_control[df_control["tt_tier"] == tier_2]
            elif k == "3rd tier":
                df_var = df_variant_1[df_variant_1["tt_tier"] == tier_3]
                df_ctl = df_control[df_control["tt_tier"] == tier_3]
            elif l == "4th tier":
                df_var = df_variant_1[df_variant_1["tt_tier"] == tier_4]
                df_ctl = df_control[df_control["tt_tier"] == tier_4]
            elif l == "5th tier":
                df_var = df_variant_1[df_variant_1["tt_tier"]== tier_5]
                df_ctl = df_control[df_control["tt_tier"] == tier_5]
           
            
            # Elasticity calculated by pct difference in no. of transactions / pct difference in average DF
            for k in range(1, num_bootstrap_samples + 1):
                print(f"Iteration {sim_counter}. Sampling with the following parameters --> vendor_group_label: {j}, price_tier:{l}, sample_num: {k}")
        
                df_ctl_sample = df_ctl.sample(frac = 1, replace = True)
                df_var_sample = df_var.sample(frac=1, replace = True)
                num_orders_ctl = df_ctl_sample["platform_order_code"].nunique()
                num_orders_var = df_var_sample["platform_order_code"].nunique()

                    # avg_df_ctl = df_ctl_sample["delivery_fee_local"].avg()
                    # avg_df_var = df_var_sample["delivery_fee_local"].avg()
                avg_df_ctl = df_ctl_sample["delivery_fee_local"].sum() / num_orders_ctl
                avg_df_var = df_var_sample["delivery_fee_local"].sum() / num_orders_var

                
                

                pct_diff_orders = float((num_orders_var - num_orders_ctl) / num_orders_ctl)
                pct_diff_avg_df = float((avg_df_var - avg_df_ctl) / avg_df_ctl)

                elasticity = pct_diff_orders / pct_diff_avg_df
                avg_df_afv_var = float(df_var_sample["delivery_fee_local"].sum()/df_var_sample["gfv_local"].sum())

                output_dict = {
                    "test_name": 'CL_20221201_R_B0_O_ElasticityCLRestaurants',
                    "vendor_group_label": j,
                    "price_tier": l,
                    "sample_num": k,
                    "elasticity": elasticity,
                    "pct_diff_orders": pct_diff_orders,
                    "pct_diff_avg_df": pct_diff_avg_df,
                    "df_afv": avg_df_afv_var
                }

                sim_results.append(output_dict)
                    
                    # Increment the sim counter
                sim_counter += 1

df_sim_results = pd.DataFrame(sim_results)

In [ ]:
df_sim_results.head()

In [ ]:
# Calculate the mean elasticity of All the clusters per price tier.
def percentile_left(x):
    return x.quantile(left_threshold)

def percentile_right(x):
    return x.quantile(right_threshold)

list_of_agg_functions = ["mean", percentile_left, percentile_right]
agg_func_selection = {"elasticity": list_of_agg_functions, "df_afv": list_of_agg_functions, "pct_diff_orders": list_of_agg_functions, "pct_diff_avg_df": list_of_agg_functions}
df_stats = df_sim_results.groupby(["test_name","price_tier", "vendor_group_label"])[["elasticity", "pct_diff_orders", "pct_diff_avg_df", "df_afv"]].agg(agg_func_selection).reset_index()
df_stats

In [ ]:
min_elas = -4.0
max_elas = 4.0
df_sim_filtered = df_sim_results[(df_sim_results['elasticity'] >= min_elas) & (df_sim_results['elasticity'] <= max_elas)]
df_sim_filtered

In [ ]:
# Filter dataset by relevant columns
df_filtered = df_sim_filtered[['price_tier', 'vendor_group_label', 'elasticity']]

# Generate a subplot for each price tier
fig, axs = plt.subplots(len(df_filtered['price_tier'].unique()), figsize=(10, 6*len(df_filtered['price_tier'].unique())))

# Iterate over price tier values and create the histograms
for i, price_tier in enumerate(df_filtered['price_tier'].unique()):
    ax = axs[i] if len(df_filtered['price_tier'].unique()) > 1 else axs
    filtered_data = df_filtered[df_filtered['price_tier'] == price_tier]
    for vendor_group_label in filtered_data['vendor_group_label'].unique():
        vendor_data = filtered_data[filtered_data['vendor_group_label'] == vendor_group_label]
        ax.hist(vendor_data['elasticity'], bins=20, alpha=0.5, label=vendor_group_label)
    ax.set_title(f'Price Tier: {price_tier}')
    ax.set_xlabel('Elasticity')
    ax.set_ylabel('Frequency')
    ax.legend()

# Adjust the space between the subplots
plt.tight_layout()

# Show plot
plt.show()